In [7]:
from src.scrapping.scrapper import AudibleScrapper
from src.graph.data_manager import DataManager
from src.graph.book_graph import BookGraph
from src.link_prediction.link_dataset import LinkDataset
from src.link_prediction.link_predictor import LinkPredictor

import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch_geometric.transforms import RandomLinkSplit

%reload_ext autoreload
%autoreload 2

Global seed set to 2276984315


Dump data from database

In [2]:
dm = DataManager()
dump = dm.get_connected_scrapped_subgraph()

Create a graph from the dump and convert it to usable tensors.

In [3]:
book_graph = BookGraph(dump)
data = book_graph.to_torch()
book_graph.plot(sample=10, figsize=(20,10))

Split the data into three sets (Train / Test / Val)

In [37]:
transform = RandomLinkSplit()
train_data, val_data, test_data = transform(data)

Create Dataset objects for each of these sets

In [38]:
train_dataset = LinkDataset(train_data, data)
val_dataset = LinkDataset(val_data, data)
test_dataset = LinkDataset(test_data, data)

Create loader objects for each sets

In [39]:
train_loader = DataLoader(train_dataset, batch_size=1)
val_loader = DataLoader(val_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

Create model for predicting links

In [40]:
# Create model for predicting links.
model = LinkPredictor(
        in_channels=4,
        hidden_channels=16,
        out_channels=1,
        embedding_size=16,
        num_nodes=data.num_nodes
        )

In [54]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type      | Params
----------------------------------------
0 | conv1     | GCNConv   | 336   
1 | conv2     | GCNConv   | 17    
2 | embedding | Embedding | 6.9 K 
----------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 20/20 [00:01<00:00, 13.79it/s, loss=31.9, v_num=21]
